## train_SGD_classifier_bigTrain
score currency based on train set


In [1]:
# SYSTEM
from os import listdir
import PyCommonFun 

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *
from collections import Counter

#GENERAL
import numpy as np
import pandas as pd

# data from Albert
from code.dataExtract import data
import code.dataExtract as de
from code import Text_reader as tr
# Read the data
d = data(folder='../../train/') # training set with all the labels
dataPath='../../train'

ImportError: No module named 'code.dataExtract'; 'code' is not a package

In [2]:
def document_from_hmtl(doc_path):
    with open(doc_path) as html_file:
        full_text = html_file.read()
        body = re.findall(r'<body[^>]*?>(.*?)</body>', full_text)
        cleantext = re.sub('<\/?span[^>]*>', ' ', body[0])
        clean_text = re.sub('<.*?>', ' ', cleantext)
        #clean_text = clean_text.lower()
        clean_text = re.sub('0.01', '1', clean_text)
        clean_text = re.sub(',', '', clean_text)
        clean_text = re.sub(r'[^a-zA-Z0-9]', ' ', clean_text)
        clean_text = re.sub('\s+', ' ', clean_text)
        return clean_text
    
def getContext(s,pattern,context=list(),width=100):
    
    
    loc=s.find(pattern)
    
    if loc == -1:
        return context
    elif loc-width < 0:
        context.append(s[:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    elif loc + width > len(s):
        context.append(s[loc-width:])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    else:
        context.append(s[loc-width:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)




### load labels and context dict.
the context dict is a dict. of dict. such as contextAllCurr = {'fileId': { ' curr' : Ngram } },
where Ngramm are part of speech centered around a currency. The contextAllCurr has been 
saved in a separate notebook (save_curencyContext_train(test)Set.ipynb)

In [2]:

# get context dict
contextAllCurr=np.load(dataPath + '/train_contextAllCurr.npy').item()

extract only currencies found in the documents, store results in foundCurrType = {'fileId': [curr]}. 

In [27]:
contextOpCurr=dict()
opCurrDict=dict()

htmlFiles=tr.get_files(dataPath + '/html')

for isin in d.docid.keys():
    
    contextOpCurr[isin]=contextAllCurr[isin][d.optCur[isin][0]]
    opCurrDict[isin]=d.optCur[isin][0]
    
    if len(contextOpCurr[isin])==0:
        contextOpCurr[isin]=document_from_hmtl(htmlFiles[d.docid[isin][0]])

In [29]:
contextOpCurr_df=pd.DataFrame(contextOpCurr.items(),columns=['ISIN','context'])
opCurr_df=pd.DataFrame(opCurrDict.items(),columns=['ISIN','opCurr'])
contextOpCurr_df=pd.merge(contextOpCurr_df,opCurr_df,on='ISIN')
contextOpCurr_df.head()



,ISIN,context,opCurr
0,XS1334556716,These Final Terms do not constitute Final Ter...,USD
1,XS1313800895,NO PROSPECTUS IS REQUIRED IN ACCORDANCE WITH ...,JPY
2,XS1378463068,Pricing Supplement The SFC takes no responsib...,HKD
3,XS1267246327,Pricing Supplement dated December 10 2015 Exe...,USD
4,US05579HAB87,[ Final Terms dated 17 December 2008 ING Bank ...,USD


In [10]:
import pickle
import numpy as np

# export as np.array
context=pickle.load(open('../context.pickle', 'rb'))
opCurr=pickle.load(open('../opCurr.pickle', 'rb'))


## classifier

In [11]:
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(context, opCurr,\
                                                                     test_size=0.3, random_state=53)

In [14]:
print(X_train[:2])

[ ' THE NOTES TO WHICH THIS PRICING SUPPLEMENT RELATES HAVE NOT BEEN REGISTERED UNDER THE SECURITIES ACT OF 1933 AS AMENDED THE SECURITIES ACT OR ANY STATE SECURITIES LAWS AND UNLESS SO REGISTERED MAY NOT BE OFFERED OR SOLD WITHIN THE UNITED STATES OR TO OR FOR THE ACCOUNT OR BENEFIT OF U S PERSONS EXCEPT PURSUANT TO AN EXEMPTION FROM OR IN A TRANSACTION NOT SUBJECT TO THE REGISTRATION REQUIREMENTS OF THE SECURITIES ACT AND APPLICABLE STATE SECURITIES LAWS Pricing Supplement dated 1 December 2015 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED as Issuer U S 20000000000 Medium Term Note Programme Issue of USD300000 Share Linked Notes due March 2016 This document constitutes the Pricing Supplement relating to the issue of Notes described herein Terms used herein shall be deemed to be defined as such for the purposes of the Conditions set forth in the Offering Circular dated 12 March 2015 as amended restated or updated This Pricing Supplement contains the final terms of the Notes an

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.externals import joblib

text_clf = Pipeline([('vect', CountVectorizer()), \
                     ('tfidf', TfidfTransformer()), \
                     ('clf', SGDClassifier(average=True))])

text_clf = text_clf.set_params(clf__alpha = 1e-04, \
                               clf__n_iter = 100, \
                               clf__penalty = 'l2', \
                               tfidf__norm = 'l2', \
                               tfidf__use_idf = False, \
                               vect__max_df = 0.4, \
                               vect__ngram_range = (1, 2))

text_clf = text_clf.fit(X_train, y_train)

y_train_pred = text_clf.predict(X_train)

AttributeError: 'list' object has no attribute 'lower'

Divide the foundCurrType in three dict:
 - single currency
 - multiple currency
 - void

In [79]:
voidCurr=dict()
singleCurr=dict()
multiCurr=dict()

for key in foundCurrType:
    if len(foundCurrType[key]) == 0:
        voidCurr[key]=foundCurrType[key]
    elif len(foundCurrType[key]) == 1:
        singleCurr[key]=foundCurrType[key]
    elif len(foundCurrType[key]) > 1:
        multiCurr[key]=foundCurrType[key]



## Create bag of word for each currency

I want to get an idea of the association between the operationnal currency and the surrounding 
words. Ideally, the additionnal currency found in the docs should show differents association.

To addess this point, I create two distinct bag of words (bow) associated to:
1) operationnal currency (consider opCurr dict)
2) additionnal  currency (consider multipleCurr dict)
and compare the bow

In [80]:
text_bow_multiCurr=[]
text_bow_OpCurr=[]

for isin in multiCurr:
    for curr in contextAllCurr[isin]:
        # do not consider operationnal currency
        if d.optCur[isin][0] == curr:
            for content in contextAllCurr[isin][curr]:
                text_bow_OpCurr.append(word_tokenize(content))
        else:
            for content in contextAllCurr[isin][curr]:
                text_bow_multiCurr.append(word_tokenize(content))

In [81]:
len(text_bow_multiCurr)

10145

In [82]:
# tokenize the text but remove ponctuations
tokenizer=RegexpTokenizer(r'\w+')
text_bow_OpCurr=tokenizer.tokenize(str(text_bow_OpCurr).lower())

tokenizer=RegexpTokenizer(r'\w+')
text_bow_multiCurr=tokenizer.tokenize(str(text_bow_multiCurr).lower())


In [83]:
# create a bag of word by counting number of occurence of each term

bow_OpCurr=Counter(text_bow_OpCurr)
len_text_OpCurr=len(text_bow_OpCurr)
# normalize the bow by dividing by length of text
for key, val in bow_OpCurr.iteritems():
    bow_OpCurr[key]=np.float(bow_OpCurr[key])/len_text_OpCurr

bow_multiCurr=Counter(text_bow_multiCurr)
len_text_multiCurr=len(text_bow_multiCurr)
# normalize the bow by dividing by length of text
for key, val in bow_multiCurr.iteritems():
    bow_multiCurr[key]=np.float(bow_multiCurr[key])/len_text_multiCurr
    

In [84]:
# use set method to find common key between the bow
key_OpCurr=set(bow_OpCurr)
key_multiCurr_set=set(bow_multiCurr)



In [89]:
len_text_OpCurr

178372

In [88]:
for key,value in sorted(bow_OpCurr.iteritems(), key=lambda (k,v):(v,k), reverse=True):
    print '%s: %s' % (key,value)

the: 0.0508151503599
date: 0.0259906263315
currency: 0.0257383445832
sprinter: 0.0247516426345
as: 0.0187865808535
usd: 0.0178895790819
certificate: 0.01735698428
trade: 0.0152434238558
to: 0.0147837104478
of: 0.0145370349606
stop: 0.014099746597
premium: 0.014099746597
on: 0.014099746597
loss: 0.014099746597
trading: 0.0132812324804
3: 0.0132419886529
eur: 0.0123842307089
time: 0.0122328616599
specified: 0.0117563294688
in: 0.0117563294688
current: 0.0112797972776
12: 0.0109378153522
conditions: 0.0100800574081
issued: 0.00931760590227
rate: 0.00870652344538
spread: 0.0084598479582
maximum: 0.0084598479582
applicable: 0.00828605386496
price: 0.00826923508174
issue: 0.00817392864351
december: 0.00817392864351
2008: 0.00817392864351
settlement: 0.00792725315632
quot: 0.00792725315632
day: 0.00792725315632
23: 0.00792725315632
9: 0.00740587087659
underlying: 0.00737223331016
final: 0.00727692687193
5: 0.00700782634046
reset: 0.00678357589756
level: 0.00678357589756
financing: 0.006783575

In [86]:
# create a dictionnary with common keys, defined as the ratio of bow_OpCurr and bow_multiCurr, which is kind of weights
bowRatio_intersect_curr=dict()
for k in key_OpCurr.intersection(key_multiCurr_set):
    bowRatio_intersect_curr[k] = bow_OpCurr[k]/bow_multiCurr[k]

In [87]:
# check if it looks ok
for key,value in sorted(bowRatio_intersect_curr.iteritems(), key=lambda (k,v):(v,k), reverse=True):
    print '%s: %s' % (key,value)

ximum: 1.74576959889
certi: 1.74576959889
770: 1.74576959889
740: 1.74576959889
38: 1.74576959889
536: 1.7344265581
490: 1.7344265581
291: 1.60769154228
te: 1.56578481656
month: 1.56578481656
minimum: 1.56578481656
current: 1.56578481656
calendar: 1.56578481656
2: 1.56578481656
1st: 1.56578481656
16: 1.56578481656
15: 1.56578481656
14: 1.56578481656
13: 1.56578481656
11: 1.56578481656
10: 1.56578481656
stop: 1.56578481656
spread: 1.56578481656
premium: 1.56578481656
on: 1.56578481656
maximum: 1.56578481656
loss: 1.56578481656
event: 1.54442298634
wm: 1.51730753995
which: 1.51730753995
viii: 1.51730753995
vii: 1.51730753995
vi: 1.51730753995
valuation: 1.51730753995
termination: 1.51730753995
st: 1.51730753995
s: 1.51730753995
reference: 1.51730753995
prepa: 1.51730753995
mi: 1.51730753995
mean: 1.51730753995
m: 1.51730753995
inconvertibili: 1.51730753995
greenwich: 1.51730753995
fixing: 1.51730753995
e: 1.51730753995
currently: 1.51730753995
company: 1.51730753995
certif: 1.51730753995

In [73]:
## save the weight

In [74]:
np.save('weight_forCurrencyScoring_bigTrain.npy', bowRatio_intersect_curr) 